# Analysis

Exploratory analysis on detecting reocurring stock price decreases.

In [10]:
import pandas as pd
import importlib

import utils.utils as utils

### Load data

Data is found at https://finance.yahoo.com/quote/NRDY/history?filter=history

In [11]:
df = pd.read_csv("data/NRDY.csv")

print(df)


           Date  Open   High   Low  Close  Adj Close   Volume
0    2023-05-02  3.86  3.860  3.43   3.51       3.51  1870900
1    2023-05-03  3.40  3.560  2.95   3.00       3.00  2183200
2    2023-05-04  2.95  3.180  2.81   2.91       2.91  1297800
3    2023-05-05  2.98  3.010  2.80   2.85       2.85  1102700
4    2023-05-08  2.84  2.955  2.78   2.91       2.91   734400
..          ...   ...    ...   ...    ...        ...      ...
248  2024-04-26  2.65  2.835  2.62   2.76       2.76   899600
249  2024-04-29  2.77  2.820  2.66   2.71       2.71   700100
250  2024-04-30  2.67  2.680  2.58   2.59       2.59   560600
251  2024-05-01  2.61  2.660  2.53   2.56       2.56   900800
252  2024-05-02  2.63  2.660  2.48   2.64       2.64   824315

[253 rows x 7 columns]


### Cleaning data

Calculating change in stock price from close of one day to close of the next day.

In [13]:
importlib.reload(utils)

df = utils.clean_df(df)

df

,Date,Open,High,Low,Close,Adj Close,Volume,percent_chg
2,2023-05-04,2.95,3.180,2.81,2.91,2.91,1297800,-0.030000
3,2023-05-05,2.98,3.010,2.80,2.85,2.85,1102700,-0.020619
4,2023-05-08,2.84,2.955,2.78,2.91,2.91,734400,0.021053
5,2023-05-09,2.88,3.105,2.84,3.10,3.10,1680400,0.065292
6,2023-05-10,3.57,3.630,2.93,3.01,3.01,2849400,-0.029032
...,...,...,...,...,...,...,...,...
248,2024-04-26,2.65,2.835,2.62,2.76,2.76,899600,0.061538
249,2024-04-29,2.77,2.820,2.66,2.71,2.71,700100,-0.018116
250,2024-04-30,2.67,2.680,2.58,2.59,2.59,560600,-0.044280
251,2024-05-01,2.61,2.660,2.53,2.56,2.56,900800,-0.011583
